In [5]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()

def get_aqi_str(aqi_key:str)->str:
    url = f'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key={aqi_key}&limit=1000&sort=datacreationdate desc&format=JSON'
    try:
        r = requests.get(url)
        r.raise_for_status()   #這樣就不用檢查 200
    except Exception as e:
        print(f'發生錯誤: {e}')

    return r.text


aqi_data:str = get_aqi_str(aqi_key=os.environ["AQI_KEY"])

In [6]:
from pydantic import BaseModel,Field

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias="county")
    aqi:int
    狀態: str = Field(alias='status')
    PM25: float = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias="longitude")





class AQI(BaseModel):
    records:list[Record]

aqi = AQI.model_validate_json(aqi_data)



ValidationError: 1 validation error for AQI
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='由本平臺呼叫環境...升資料索取權限', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid

In [ ]:
#驗證錯誤，pm2.5有空字串，無法轉換為float

In [9]:
from pydantic import BaseModel,Field,field_validator

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias="county")
    aqi: float | None
    狀態: str = Field(alias='status')
    PM25: float | None = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias="longitude")

    @field_validator('PM25','aqi',mode='before')   #先做下面的手動轉換 在做上面的自動轉換
    @classmethod
    def pm25_for_empty_string(cls,v:str)->float:
        if v =="":
            return None
        else:
            return float(v)



class AQI(BaseModel):
    records:list[Record]


aqi = AQI.model_validate_json(aqi_data)

In [ ]:
aqi.records

In [ ]:
aqi.model_dump()

In [ ]:
aqi.model_dump()['records']

In [3]:
from pydantic import BaseModel,Field,field_validator,field_serializer
from datetime import datetime

class Record(BaseModel):
    日期: datetime = Field(alias='datacreationdate')
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias="county")
    aqi: float | None
    狀態: str = Field(alias='status')
    PM25: float | None = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias="longitude")

    @field_validator('PM25','aqi',mode='before')   #先做下面的手動轉換 在做上面的自動轉換
    @classmethod
    def pm25_for_empty_string(cls,v:str)->float:
        if v =="":
            return None
        else:
            return float(v)
        
    @field_serializer('緯度','經度')
    def four_digits(self,value:float)->float:
        return round(value,ndigits=4)
    
    @field_serializer('日期')
    def datetime_to_str(self,date:datetime)->str:
        return date.strftime('%Y-%m-%d %H-%M:00')



class AQI(BaseModel):
    records:list[Record]


aqi = AQI.model_validate_json(aqi_data)

ValidationError: 1 validation error for AQI
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='由本平臺呼叫環境...升資料索取權限', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid

In [4]:
aqi_data:list[dict] = aqi.model_dump()['records']
for aqi_item in aqi_data:
    print(aqi_item)

NameError: name 'aqi' is not defined